<a href="https://colab.research.google.com/github/bigyansapkota97/Covid-vs-Stock-Price/blob/main/Python_project_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install alpha_vantage

In [ ]:
import pandas as pd

url_conf_cases = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
url_deaths = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"

df_conf_cases_initial = pd.read_csv(url_conf_cases)
df_deaths_initial = pd.read_csv(url_deaths)